In [ ]:
# 🧪 Python Kodēšana 2 – Tests (1.–4. nodarbība) - Timer Cell
from IPython.display import display, clear_output
import time
from datetime import datetime
import requests
import ipywidgets as widgets

# Global state (shared with quiz cell)
submitted = False
results = [None] * 8  # Initialize for 8 questions
attempts = [0] * 8

# Google Forms konfigurācija
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdTSrnEP9GQajJRRCKqQMbwWGS_TL3o3B4T_fequkWmUZQCCg/formResponse"
entry_ids = {
    "Students": "entry.1649720525",
    "Datums": "entry.667959859",
    "Rezultats": "entry.940605140",
    "1": "entry.213159259",
    "2": "entry.267050949",
    "3": "entry.418893037",
    "4": "entry.1447996441",
    "5": "entry.1914428828",
    "6": "entry.1728217231",
    "7": "entry.1875638404",
    "8": "entry.907839017"
}

# Shared state for submission
name_input = widgets.Text(description='Vārds:', placeholder='Ievadi savu vārdu')
questions_data = [
    ("Ko dara pygame.init()?", [
        "Inicializē visus importētos pygame moduļus",
        "Palaiž spēles ciklu",
        "Ielādē spēles logu",
        "Izveido galveno varoņa sprite objektu"
    ], 0),
    ("Kura metode tiek izmantota, lai uzzīmētu taisnstūri pygame logā?", [
        "pygame.draw.rect()",
        "pygame.screen.rectangle()",
        "pygame.create.rect()",
        "pygame.render.rect()"
    ], 0),
    ("Kā tiek atjaunināts logs pygame pēc zīmēšanas?", [
        "pygame.display.flip()",
        "pygame.update.display()",
        "pygame.screen.refresh()",
        "pygame.render.frame()"
    ], 0),
    ("Ko pārbauda event.type == pygame.QUIT?", [
        "Vai lietotājs ir nospiedis loga aizvēršanas pogu",
        "Vai lietotājs nospieda taustiņu Enter",
        "Vai spēlei jāsākas no jauna",
        "Vai spēlētājs zaudēja"
    ], 0),
    ("Kurš modulis tiek izmantots, lai veidotu GUI lietotnes Python valodā?", [
        "tkinter",
        "pygame.gui",
        "guimodule",
        "tkbuilder"
    ], 0),
    ("Kurš tkinter logelements tiek izmantots, lai saņemtu tekstu no lietotāja?", [
        "Entry",
        "Label",
        "Button",
        "Canvas"
    ], 0),
    ("Ko dara mainloop() tkinter lietotnē?", [
        "Palaiž GUI notikumu ciklu",
        "Pieslēdzas datubāzei",
        "Atjauno Python vidi",
        "Izveido loga izvietojumu"
    ], 0),
    ("Kurš tkinter logelements ļauj zīmēt figūras kā taisnstūrus un tekstu?", [
        "Canvas",
        "Frame",
        "Box",
        "Entry"
    ], 0)
]

# Shuffle questions
import random
random.shuffle(questions_data)

# Timer display
refresh_button = widgets.Button(description="Atjaunot taimeri", button_style='warning')

def refresh_timer(_):
    clear_output(wait=True)
    print(f"Atlikušais laiks: {timer_text}")
    display(refresh_button)

refresh_button.on_click(refresh_timer)

# Submission function
def send_to_google_form(_):
    global submitted
    if submitted:
        return
    submitted = True
    student_name = name_input.value.strip() or "Nezināms"
    total = sum(r if r else 0 for r in results)
    final_score = round(total)
    now = datetime.now().strftime("%Y-%m-%d %H:%M")
    form_data = {
        entry_ids["Students"]: student_name,
        entry_ids["Datums"]: now,
        entry_ids["Rezultats"]: str(final_score) + "%"
    }
    for i, (q, _, _) in enumerate(questions_data):
        question_entry = entry_ids[str(i+1)]
        response = f"{q} (Mēģinājumi: {attempts[i]}) – {results[i] if results[i] is not None else 0} punkti"
        form_data[question_entry] = response
    try:
        response = requests.post(form_url, data=form_data)
        if response.status_code == 200:
            print("Rezultāti veiksmīgi nosūtīti uz Google Formu!")
        else:
            print(f"Kļūda sūtot datus (statuss: {response.status_code})")
    except Exception as e:
        print(f"Kļūda sūtot datus: {e}")

# Timer loop
timer_text = "40:00"
minutes = 40
seconds = minutes * 60
start_time = time.time()
try:
    while seconds > 0 and not submitted:
        elapsed = time.time() - start_time
        remaining = max(0, seconds - int(elapsed))
        mins, secs = divmod(remaining, 60)
        timer_text = f"{mins:02d}:{secs:02d}"
        clear_output(wait=True)
        print(f"Atlikušais laiks: {timer_text}")
        display(refresh_button)
        time.sleep(1)
        if time.time() - start_time >= seconds:
            break
    if not submitted:
        timer_text = "Laiks beidzies! Rezultāti nosūtīti."
        clear_output(wait=True)
        print(timer_text)
        send_to_google_form(None)
except Exception as e:
    print(f"Timer error: {e}")


In [ ]:
# 🧪 Python Kodēšana 2 – Tests (1.–4. nodarbība) - HTML Quiz Cell
from IPython.display import display, HTML
from datetime import datetime
import requests
import json

# Use shared state from timer cell (submitted, results, attempts, questions_data, name_input, form_url, entry_ids)

# Generate HTML quiz
html_content = """
<style>
    .quiz-container { font-family: Arial, sans-serif; max-width: 800px; margin: 20px; }
    .question { margin-bottom: 20px; }
    .question p { font-weight: bold; }
    .options { margin-left: 20px; }
    .feedback { color: #333; font-style: italic; }
    .submit-btn, .download-btn, .send-btn { padding: 10px 20px; margin: 10px 5px; cursor: pointer; }
    .submit-btn { background-color: #28a745; color: white; border: none; }
    .download-btn { background-color: #17a2b8; color: white; border: none; }
    .send-btn { background-color: #007bff; color: white; border: none; }
    .disabled { background-color: #ccc; cursor: not-allowed; }
</style>
<div class='quiz-container' id='quiz'>
    <input type='text' id='name-input' placeholder='Ievadi savu vārdu' style='padding: 8px; margin-bottom: 20px; width: 200px;'/>
"""

for i, (question, options, correct_index) in enumerate(questions_data):
    # Shuffle options
    paired = list(enumerate(options))
    random.shuffle(paired)
    shuffled_indexes, shuffled_options = zip(*paired)
    correct_shuffled_index = shuffled_indexes.index(correct_index)

    html_content += f"""
    <div class='question' id='q{i}'>
        <p>{i+1}. {question}</p>
        <div class='options'>
    """
    for j, option in enumerate(shuffled_options):
        html_content += f"""
            <label><input type='radio' name='q{i}' value='{j}' onclick='enableCheck({i})'> {option}</label><br>
        """
    html_content += f"""
        </div>
        <button id='check-btn-{i}' onclick='checkAnswer({i}, {correct_shuffled_index})' disabled>Pārbaudīt</button>
        <p id='feedback-{i}' class='feedback'></p>
    </div>
    """

html_content += f"""
    <div id='submit-section' style='display: none;'>
        <button class='submit-btn' id='submit-btn' onclick='calculateScore()'>Iesniegt rezultātu</button>
        <button class='download-btn' id='download-btn' onclick='downloadResult()' style='display: none;'>⬇️ Lejupielādēt rezultātu</button>
        <button class='send-btn' id='send-btn' onclick='sendToGoogleForm()' style='display: none;'>📤 Sūtīt uz Google Formu</button>
    </div>
</div>
<script>
    let attempts = Array(8).fill(0);
    let results = Array(8).fill(null);
    let submitted = false;
    let questions = {json.dumps([q for q, _, _ in questions_data])};

    function enableCheck(index) {{
        document.getElementById(`check-btn-${{index}}`).disabled = false;
    }}

    function checkAnswer(index, correctIndex) {{
        if (submitted) return;
        attempts[index]++;
        let selected = document.querySelector(`input[name='q${{index}}']:checked`);
        let feedback = document.getElementById(`feedback-${{index}}`);
        if (selected && parseInt(selected.value) === correctIndex) {{
            let penalty = 5 * (attempts[index] - 1);
            let score = Math.max(12.5 - penalty, 0);
            results[index] = score;
            feedback.innerHTML = '✔️ Pareizi!';
            feedback.style.color = 'green';
            disableQuestion(index);
            if (results.every(r => r !== null)) {{
                document.getElementById('submit-section').style.display = 'block';
            }}
        }} else {{
            feedback.innerHTML = '❌ Nepareizi. Mēģini vēlreiz.';
            feedback.style.color = 'red';
        }}
    }}

    function disableQuestion(index) {{
        let radios = document.getElementsByName(`q${{index}}`);
        for (let radio of radios) radio.disabled = true;
        document.getElementById(`check-btn-${{index}}`).disabled = true;
    }}

    function calculateScore() {{
        if (submitted) return;
        let total = results.reduce((sum, r) => sum + (r || 0), 0);
        let finalScore = Math.round(total);
        let status = finalScore >= 70 ? '✅ Nokārtots!' : '❌ Nenokārtots.';
        let color = finalScore >= 70 ? 'green' : 'red';
        let resultDiv = document.createElement('div');
        resultDiv.innerHTML = `<h3>Rezultāts: <span style='color:${{color}}'>${{finalScore}}%</span> – ${{status}}</h3>`;
        document.getElementById('quiz').prepend(resultDiv);
        document.getElementById('submit-btn').style.display = 'none';
        document.getElementById('download-btn').style.display = 'inline-block';
        document.getElementById('send-btn').style.display = 'inline-block';
    }}

    function downloadResult() {{
        if (submitted) return;
        let studentName = document.getElementById('name-input').value.trim() || 'Nezināms';
        let total = results.reduce((sum, r) => sum + (r || 0), 0);
        let finalScore = Math.round(total);
        let now = new Date().toISOString().slice(0, 16).replace('T', ' ');
        let content = `Students: ${{studentName}}\nDatums: ${{now}}\nRezultāts: ${{finalScore}}%\n\n`;
        for (let i = 0; i < 8; i++) {{
            content += `${{i+1}}. ${{questions[i]}} (Mēģinājumi: ${{attempts[i]}}) – ${{results[i] || 0}} punkti\n`;
        }}
        let blob = new Blob([content], {{type: 'text/plain'}});
        let a = document.createElement('a');
        a.href = URL.createObjectURL(blob);
        a.download = `rezultats_${{studentName.replace(/ /g, '_')}}.txt`;
        a.click();
    }}

    function sendToGoogleForm() {{
        if (submitted) return;
        submitted = true;
        let studentName = document.getElementById('name-input').value.trim() || 'Nezināms';
        let total = results.reduce((sum, r) => sum + (r || 0), 0);
        let finalScore = Math.round(total);
        let now = new Date().toISOString().slice(0, 16).replace('T', ' ');
        let formData = {{
            'entry.1649720525': studentName,
            'entry.667959859': now,
            'entry.940605140': `${{finalScore}}%`, 
            'entry.213159259': `${{questions[0]}} (Mēģinājumi: ${{attempts[0]}}) – ${{results[0] || 0}} punkti`, 
            'entry.267050949': `${{questions[1]}} (Mēģinājumi: ${{attempts[1]}}) – ${{results[1] || 0}} punkti`, 
            'entry.418893037': `${{questions[2]}} (Mēģinājumi: ${{attempts[2]}}) – ${{results[2] || 0}} punkti`, 
            'entry.1447996441': `${{questions[3]}} (Mēģinājumi: ${{attempts[3]}}) – ${{results[3] || 0}} punkti`, 
            'entry.1914428828': `${{questions[4]}} (Mēģinājumi: ${{attempts[4]}}) – ${{results[4] || 0}} punkti`, 
            'entry.1728217231': `${{questions[5]}} (Mēģinājumi: ${{attempts[5]}}) – ${{results[5] || 0}} punkti`, 
            'entry.1875638404': `${{questions[6]}} (Mēģinājumi: ${{attempts[6]}}) – ${{results[6] || 0}} punkti`, 
            'entry.907839017': `${{questions[7]}} (Mēģinājumi: ${{attempts[7]}}) – ${{results[7] || 0}} punkti`
        }};
        fetch('{form_url}', {{
            method: 'POST',
            mode: 'no-cors',
            headers: {{ 'Content-Type': 'application/x-www-form-urlencoded' }},
            body: new URLSearchParams(formData).toString()
        }}).then(() => {{
            let feedback = document.createElement('p');
            feedback.innerHTML = '<h4 style="color:green;">Rezultāti veiksmīgi nosūtīti uz Google Formu!</h4>';
            document.getElementById('quiz').prepend(feedback);
            disableQuiz();
        }}).catch(e => {{
            let feedback = document.createElement('p');
            feedback.innerHTML = `<h4 style="color:red;">Kļūda sūtot datus: ${{e}}</h4>`;
            document.getElementById('quiz').prepend(feedback);
        }});
    }}

    function disableQuiz() {{
        document.getElementById('name-input').disabled = true;
        document.getElementById('submit-btn').disabled = true;
        document.getElementById('download-btn').disabled = true;
        document.getElementById('send-btn').disabled = true;
        document.getElementById('submit-btn').classList.add('disabled');
        document.getElementById('download-btn').classList.add('disabled');
        document.getElementById('send-btn').classList.add('disabled');
        for (let i = 0; i < 8; i++) {{
            let radios = document.getElementsByName(`q${{i}}`);
            for (let radio of radios) radio.disabled = true;
            document.getElementById(`check-btn-${{i}}`).disabled = true;
            document.getElementById(`check-btn-${{i}}`).classList.add('disabled');
        }}
    }}
</script>
"""

# Display quiz
display(HTML(html_content))
